<a href="https://colab.research.google.com/github/ruval195/Pattern_Recognition/blob/main/Examen_1erParcial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reconocimeinto de Patrones
## Diego Francisco Ruvalcaba Loredo
## Examen Practico 1er Parcial
### "Tenemos nuevos datos sobre ataques cardiacos, ¿Serias tan amable de usar tus habilidades en ML para crear un predictor de dichos ataques?. Necesitamos que tenga la mejor exactitud posible, no hay problema si nos entregas varios modelos y sus respectivos desempeños."


### Librerias

In [142]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error


### Codificacion de datos

Descargar csv

In [143]:
!wget 'https://raw.githubusercontent.com/ruval195/Pattern_Recognition/refs/heads/main/Data_set_ha/data.csv'
data = pd.read_csv('data.csv')

--2025-03-01 20:02:30--  https://raw.githubusercontent.com/ruval195/Pattern_Recognition/refs/heads/main/Data_set_ha/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 316971 (310K) [text/plain]
Saving to: ‘data.csv.18’

data.csv.18         100%[===================>] 309.54K  --.-KB/s    in 0.02s   

2025-03-01 20:02:31 (18.3 MB/s) - ‘data.csv.18’ saved [316971/316971]



Eliminar columnas inecesarias

In [144]:
drop_cols = ["id","ever_married", "work_type", "Residence_type", "stroke"]
data2 = data.drop(columns=drop_cols, errors='ignore')
data2 = pd.get_dummies(data2, drop_first=True)
# Elimino el valor "NaN" por un valor entero 0 por que es desconocido
data2["bmi"] = pd.to_numeric(data2["bmi"], errors="coerce").fillna(0)
data2.head()

,age,hypertension,heart_disease,avg_glucose_level,bmi,gender_Male,gender_Other,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,67.0,0,1,228.69,36.6,True,False,True,False,False
1,61.0,0,0,202.21,0.0,False,False,False,True,False
2,80.0,0,1,105.92,32.5,True,False,False,True,False
3,49.0,0,0,171.23,34.4,False,False,False,False,True
4,79.0,1,0,174.12,24.0,False,False,False,True,False


Definir varibles de entrada y salida

In [145]:
x = np.asanyarray(data2)
y = np.asanyarray(data[['stroke']]).ravel()

# Dividir Train/Test
xtrain, xtest, ytrain, ytest = train_test_split(x,y, test_size=0.2, random_state=42)

### Modelo de Regresion logistica

Entrenamiento

In [146]:
# Crear modelo y entrenar
model_reg = Pipeline([
    ('scaler', StandardScaler()),
    ('logit', LogisticRegression(solver='lbfgs')) ])

# Crear modelo y entrenar
model_reg.fit(xtrain,ytrain)

# Calcular Score
print('Train: ', model_reg.score(xtrain,ytrain))
print('Test: ', model_reg.score(xtest,ytest))

Train:  0.9537671232876712
Test:  0.9393346379647749


Metricas

In [153]:
ytrain_pred = model_reg.predict(xtrain)
ytest_pred = model_reg.predict(xtest)

print('Accuracy train:', accuracy_score(ytrain, ytrain_pred))
print('Accuracy test:', accuracy_score(ytest, ytest_pred))

ypred = model_reg.predict(xtest)
mse = mean_squared_error(ytest, ypred)
print("MSE:", mse)

Accuracy train: 0.9537671232876712
Accuracy test: 0.9393346379647749
MSE: 0.060665362035225046


### MLP

Entrenamiento

In [148]:
# Se modificaron las variable de las layers y el batch size y el modelo no mejoraba las prediccines
model_mlp = Pipeline([
    ('scaler', StandardScaler()),
    ('mlp', MLPRegressor(hidden_layer_sizes=(1024,512,256), solver='adam', activation='relu',
                     validation_fraction=0.1,early_stopping=True,
                     batch_size=125, verbose=1))
])
model_mlp.fit(xtrain, ytrain.ravel())

Iteration 1, loss = 0.03397090
Validation score: 0.057947
Iteration 2, loss = 0.02084533
Validation score: 0.030359
Iteration 3, loss = 0.02020305
Validation score: 0.080088
Iteration 4, loss = 0.01974956
Validation score: 0.101908
Iteration 5, loss = 0.01989658
Validation score: 0.075861
Iteration 6, loss = 0.01951393
Validation score: 0.089533
Iteration 7, loss = 0.01977982
Validation score: 0.062692
Iteration 8, loss = 0.01941578
Validation score: 0.031333
Iteration 9, loss = 0.01947825
Validation score: 0.100567
Iteration 10, loss = 0.01883009
Validation score: 0.047749
Iteration 11, loss = 0.01880186
Validation score: 0.063968
Iteration 12, loss = 0.01884943
Validation score: 0.021761
Iteration 13, loss = 0.01859918
Validation score: 0.050443
Iteration 14, loss = 0.01890198
Validation score: -0.032469
Iteration 15, loss = 0.01917797
Validation score: 0.091455
Validation score did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


Pipeline(steps=[('scaler', StandardScaler()),
                ('mlp',
                 MLPRegressor(batch_size=125, early_stopping=True,
                              hidden_layer_sizes=(1024, 512, 256),
                              verbose=1))])

Metricas

In [154]:
## El modelo se encuentra subentrenado
print('Train: ', model_mlp.score(xtrain, ytrain.ravel()))
print('Test: ', model_mlp.score(xtest, ytest.ravel()))

ypred = model_mlp.predict(xtest)
mse = mean_squared_error(ytest, ypred)
print("MSE:", mse)

Train:  0.13414034287843146
Test:  0.11946427583012553
MSE: 0.05017739506071028


### SVM

Entrenamiento

In [192]:
model_svr = SVR(C=10, kernel='rbf', gamma=0.1)
model_svr.fit(xtrain, ytrain.ravel())

SVR(C=10, gamma=0.1)

Metricas

In [193]:
## El modelo con todas las variaciones llega a estar sobreentrenado y al modificar los parametros del mismo llega a ser subentrenado
print('Train: ', model_svr.score(xtrain, ytrain.ravel()))
print('Test: ', model_svr.score(xtest, ytest.ravel()))

ypred = model_svr.predict(xtest)
mse = mean_squared_error(ytest, ypred)
print("MSE:", mse)

Train:  0.7994015096280287
Test:  -0.1299290436392344
MSE: 0.06438909229572422


###Conclusion

El hecho de que la regresión logística haya sido el modelo más efectivo sugiere que el problema tiene relaciones simples y lineales entre las variables, lo que hace que un modelo más sencillo sea adecuado.
El MLP, al estar subentrenado, podría beneficiarse de ajustes en su arquitectura o de más épocas de entrenamiento, mientras que el SVC, al sobreajustarse, indica que necesita un mejor manejo de la regularización para evitar el sobreentrenamiento y mejorar su capacidad de generalización. En general, es necesario ajustar los hiperparámetros de ambos modelos más complejos para mejorar su rendimiento.